In [1]:
import sys
import torch
sys.path.append('..')
from models.encoder import TransformerEncoder

In [3]:
encoder = TransformerEncoder(
    input_dim = 2,
    d_model = 6,
    nhead = 2,
    nhid = 8,
    nlayers = 2,
    dropout = 0.1
)

In [4]:
inputs = torch.Tensor([
    [1,2],
    [3,4]
])
output = encoder(inputs)


In [5]:
output.shape

torch.Size([2, 6])